# Token Classification using BERT models
## This notebook outlines the usage of BERT models in Token Classification task (Named Entity Recognition)

# Fine-tuning a model on a token NER classification task

Fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

In [1]:
task = "ner" 
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

In [2]:
from datasets import load_dataset, load_metric

Using the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). 

In [3]:
datasets = load_dataset("conll2003")

Generating train split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3454 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3454
    })
})

To access an actual element, you need to select a split first, then give an index:

In [5]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [7]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,12130,"[Iran, has, warned, Germany, that, bilateral, relations, could, suffer, if, it, pays, heed, to, the, testimony, of, Banisadr, ,, an, architect, of, Iran, 's, Islamic, revolution, who, has, been, a, sworn, enemy, of, Tehran, since, he, fell, from, favour, after, a, year, as, president, .]","[NNP, VBZ, VBN, NNP, IN, JJ, NNS, MD, VB, IN, PRP, VBZ, NN, TO, DT, NN, IN, NNP, ,, DT, NN, IN, NNP, POS, JJ, NN, WP, VBZ, VBN, DT, VBN, NN, IN, NNP, IN, PRP, VBD, IN, NN, IN, DT, NN, IN, NN, .]","[B-NP, B-VP, I-VP, B-NP, B-SBAR, B-NP, I-NP, B-VP, I-VP, B-SBAR, B-NP, B-VP, B-NP, B-PP, B-NP, I-NP, B-PP, B-NP, O, B-NP, I-NP, B-PP, B-NP, B-NP, I-NP, I-NP, B-NP, B-VP, I-VP, B-NP, I-NP, I-NP, B-PP, B-NP, B-SBAR, B-NP, B-VP, B-PP, B-NP, B-PP, B-NP, I-NP, B-PP, B-NP, O]","[B-LOC, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, O, O, O, O, B-LOC, O, B-MISC, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O]"
1,10780,"[Strasbourg, 4, 1, 0, 3, 2, 7, 3]","[NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, O, O, O, O, O, O]"
2,13408,"[Egypt, police, catch, ancient, manuscript, thieves, .]","[NNP, NN, NN, JJ, NN, NNS, .]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, O]","[B-LOC, O, O, O, O, O, O]"
3,5734,[Halftime],[VB],[B-VP],[O]
4,3382,"[RALLYING, -, KANKKUNEN, IN, COMMAND, AS, MCRAE, ROLLS, OUT, .]","[VBG, :, NNP, IN, NNP, NNP, NNP, NNP, NNP, .]","[B-VP, O, B-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, O]","[O, O, B-PER, O, O, O, B-PER, O, O, O]"
5,1667,"[Analysts, had, on, average, expected, a, pre-abnormals, profit, of, A$, 1.343, billion, .]","[NNS, VBD, IN, JJ, JJ, DT, JJ, NN, IN, $, CD, CD, .]","[B-NP, B-VP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, O]","[O, O, O, O, O, O, O, O, O, B-MISC, O, O, O]"
6,9260,"[Price, per, bushel, for, 14-pct, protein, dark, northern, spring, ,, durum, and, white, wheats, ,, in, dollars, per, bushel, :]","[NN, IN, NN, IN, JJ, NN, JJ, JJ, NN, ,, NN, CC, JJ, NNS, ,, IN, NNS, IN, NN, :]","[B-NP, B-PP, B-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, I-NP, O, B-PP, B-NP, B-PP, B-NP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,1680,"["", If, they, 're, saying, at, least, 20, percent, ,, then, their, internal, forecasts, are, probably, saying, 25, or, 30, percent, ,, "", said, one, Sydney, media, analyst, who, declined, to, be, named, .]","["", IN, PRP, VBP, VBG, IN, JJS, CD, NN, ,, RB, PRP$, JJ, NNS, VBP, RB, VBG, CD, CC, CD, NN, ,, "", VBD, CD, NNP, NNS, NN, WP, VBD, TO, VB, VBN, .]","[O, B-SBAR, B-NP, B-VP, I-VP, B-NP, I-NP, I-NP, I-NP, O, B-ADVP, B-NP, I-NP, I-NP, B-VP, I-VP, I-VP, B-NP, I-NP, I-NP, I-NP, O, O, B-VP, B-NP, I-NP, I-NP, I-NP, B-NP, B-VP, I-VP, I-VP, I-VP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O]"
8,1796,"[New, York, Dow, Jones, industrial, average, --, 5,778.00, (, May, 22/96, )]","[NNP, NNP, NNP, NNP, JJ, NN, :, CD, (, NNP, CD, )]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, O, B-NP, O, B-NP, I-NP, O]","[B-LOC, I-LOC, B-MISC, I-MISC, O, O, O, O, O, O, O, O]"
9,12521,"[DETROIT, 47, 86, .353, 27, 1/2]","[VBG, CD, CD, CD, CD, CD]","[B-VP, B-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, O, O, O, O, O]"


## Preprocessing the data

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [11]:
label_all_tokens = True

In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [13]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [14]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [15]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [17]:
pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 197.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=973f43f8ef9b4ad921edc63f11c90d61e54ea8cd74802c6c376ba5df35818280
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [18]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [19]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


We can now finetune our model by just calling the `train` method:

In [21]:
import os
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: pos_tags, id, chunk_tags, tokens, ner_tags. If pos_tags, id, chunk_tags, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14042
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.221000,0.065658,0.917869,0.925159,0.921500,0.981651
2,0.046600,0.060063,0.920827,0.936794,0.928742,0.983446
3,0.026800,0.058320,0.931670,0.944177,0.937882,0.985496


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: pos_tags, id, chunk_tags, tokens, ner_tags. If pos_tags, id, chunk_tags, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3251
  Batch size = 16
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in test-ner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1000/tokenizer_config.json
Special tokens 

TrainOutput(global_step=2634, training_loss=0.07857917426839957, metrics={'train_runtime': 359.9893, 'train_samples_per_second': 117.02, 'train_steps_per_second': 7.317, 'total_flos': 1019599557281136.0, 'train_loss': 0.07857917426839957, 'epoch': 3.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [22]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: pos_tags, id, chunk_tags, tokens, ner_tags. If pos_tags, id, chunk_tags, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3251
  Batch size = 16


{'eval_loss': 0.058320172131061554,
 'eval_precision': 0.9316701622695662,
 'eval_recall': 0.9441772010291979,
 'eval_f1': 0.9378819868874319,
 'eval_accuracy': 0.9854957345068073,
 'eval_runtime': 8.6119,
 'eval_samples_per_second': 377.5,
 'eval_steps_per_second': 23.688,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [23]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: pos_tags, id, chunk_tags, tokens, ner_tags. If pos_tags, id, chunk_tags, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3251
  Batch size = 16


{'LOC': {'precision': 0.9601820250284414,
  'recall': 0.9671504965622613,
  'f1': 0.9636536631779258,
  'number': 2618},
 'MISC': {'precision': 0.8190401258851299,
  'recall': 0.8456539398862714,
  'f1': 0.8321342925659474,
  'number': 1231},
 'ORG': {'precision': 0.9042857142857142,
  'recall': 0.9236381322957199,
  'f1': 0.9138594802694899,
  'number': 2056},
 'PER': {'precision': 0.9727958046542118,
  'recall': 0.978246539222149,
  'f1': 0.9755135579293344,
  'number': 3034},
 'overall_precision': 0.9316701622695662,
 'overall_recall': 0.9441772010291979,
 'overall_f1': 0.9378819868874319,
 'overall_accuracy': 0.9854957345068073}